# Mbinu za Uangalizi na Transformers

Changamoto kubwa ya mitandao ya kurudiarudia (recurrent networks) ni kwamba maneno yote katika mfuatano yana athari sawa kwenye matokeo. Hii husababisha utendaji usiofaa katika mifano ya kawaida ya LSTM encoder-decoder kwa kazi za mfuatano hadi mfuatano, kama Utambuzi wa Vitu Vilivyotajwa (Named Entity Recognition) na Tafsiri ya Mashine. Kwa kweli, maneno maalum katika mfuatano wa pembejeo mara nyingi yana athari kubwa zaidi kwenye matokeo ya mfuatano kuliko mengine.

Fikiria mfano wa mfuatano hadi mfuatano, kama tafsiri ya mashine. Inatekelezwa na mitandao miwili ya kurudiarudia, ambapo mtandao mmoja (**encoder**) unakunjwa mfuatano wa pembejeo kuwa hali iliyofichwa, na mwingine, **decoder**, unafungua hali hiyo iliyofichwa kuwa matokeo yaliyotafsiriwa. Tatizo la mbinu hii ni kwamba hali ya mwisho ya mtandao itakuwa na ugumu wa kukumbuka mwanzo wa sentensi, hivyo kusababisha ubora duni wa mfano kwa sentensi ndefu.

**Mbinu za Uangalizi** hutoa njia ya kupima athari ya muktadha wa kila vector ya pembejeo kwenye kila utabiri wa matokeo wa RNN. Njia inavyotekelezwa ni kwa kuunda njia za mkato kati ya hali za kati za RNN ya pembejeo na RNN ya matokeo. Kwa njia hii, tunapozalisha alama ya matokeo $y_t$, tutazingatia hali zote zilizofichwa za pembejeo $h_i$, kwa kutumia viwango tofauti vya uzito $\alpha_{t,i}$.

![Picha inayoonyesha mfano wa encoder/decoder na safu ya uangalizi wa kuongeza](../../../../../translated_images/sw/encoder-decoder-attention.7a726296894fb567.webp)
*Mfano wa encoder-decoder na mbinu ya uangalizi wa kuongeza katika [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), iliyotajwa kutoka [blogu hii](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matriki ya uangalizi $\{\alpha_{i,j}\}$ itaonyesha kiwango ambacho maneno fulani ya pembejeo yanachangia katika uzalishaji wa neno fulani katika mfuatano wa matokeo. Hapo chini kuna mfano wa matriki kama hiyo:

![Picha inayoonyesha mpangilio wa mfano uliopatikana na RNNsearch-50, kutoka Bahdanau - arviz.org](../../../../../translated_images/sw/bahdanau-fig3.09ba2d37f202a6af.webp)

*Picha iliyotolewa kutoka [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Mbinu za uangalizi zinahusika na sehemu kubwa ya hali ya juu ya sasa au karibu na hali ya juu katika Usindikaji wa Lugha Asilia. Hata hivyo, kuongeza uangalizi huongeza sana idadi ya vigezo vya mfano, jambo ambalo limesababisha changamoto za kupanua RNNs. Kizuizi kikuu cha kupanua RNNs ni kwamba asili ya kurudiarudia ya mifano inafanya iwe changamoto kuunda vikundi na kuendesha mafunzo kwa sambamba. Katika RNN, kila kipengele cha mfuatano kinahitaji kusindika kwa mpangilio wa mfuatano, jambo ambalo lina maana kuwa haiwezi kufanywa kwa urahisi kwa sambamba.

Kupitishwa kwa mbinu za uangalizi pamoja na kizuizi hiki kumesababisha kuundwa kwa mifano ya Transformer, ambayo sasa ni hali ya juu ya sanaa (State of the Art) tunayoijua na kuitumia leo, kutoka BERT hadi OpenGPT3.

## Mifano ya Transformer

Badala ya kupeleka muktadha wa kila utabiri wa awali kwenye hatua inayofuata ya tathmini, **mifano ya transformer** hutumia **usimbaji wa nafasi** na **uangalizi** ili kunasa muktadha wa pembejeo fulani ndani ya dirisha lililotolewa la maandishi. Picha hapa chini inaonyesha jinsi usimbaji wa nafasi na uangalizi vinavyoweza kunasa muktadha ndani ya dirisha lililotolewa.

![GIF ya michoro inayoonyesha jinsi tathmini zinavyofanywa katika mifano ya transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Kwa kuwa kila nafasi ya pembejeo inahusishwa kwa uhuru na kila nafasi ya matokeo, transformers zinaweza kufanya kazi kwa sambamba vizuri zaidi kuliko RNNs, jambo ambalo linawezesha mifano mikubwa zaidi na yenye lugha ya kuelezea zaidi. Kila kichwa cha uangalizi kinaweza kutumika kujifunza mahusiano tofauti kati ya maneno, jambo ambalo linaboresha kazi za Usindikaji wa Lugha Asilia.

## Kujenga Mfano Rahisi wa Transformer

Keras haina safu ya Transformer iliyojengwa ndani, lakini tunaweza kujenga yetu wenyewe. Kama ilivyokuwa awali, tutazingatia uainishaji wa maandishi wa seti ya data ya AG News, lakini ni muhimu kutaja kwamba mifano ya Transformer inaonyesha matokeo bora zaidi katika kazi ngumu zaidi za NLP.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Tabaka mpya katika Keras yanapaswa kurithi darasa la `Layer`, na kutekeleza njia ya `call`. Hebu tuanze na tabaka la **Positional Embedding**. Tutatumia [baadhi ya msimbo kutoka kwa nyaraka rasmi za Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Tutadhania kwamba tunajaza mfuatano wote wa ingizo hadi urefu wa `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Tabaka hili lina tabaka mbili za `Embedding`: moja kwa kuweka tokeni (kwa njia tuliyojadili awali) na nyingine kwa nafasi za tokeni. Nafasi za tokeni zinaundwa kama mlolongo wa nambari za asili kutoka 0 hadi `maxlen` kwa kutumia `tf.range`, kisha hupitishwa kupitia tabaka ya embedding. Vekta mbili za embedding zinazotokana zinaongezwa, na kutoa uwakilishi wa pembejeo ulio na nafasi ya umbo `maxlen`$\times$`embed_dim`.

Sasa, hebu tekeleze kizuizi cha transformer. Kitachukua matokeo ya tabaka ya embedding iliyofafanuliwa hapo awali:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Sasa, tuko tayari kufafanua mfano kamili wa transformer: 

Transformer hutumia `MultiHeadAttention` kwa pembejeo iliyowekwa nafasi ili kuzalisha vector ya umakini yenye dimension `maxlen`$\times$`embed_dim`, ambayo kisha huchanganywa na pembejeo na kusawazishwa kwa kutumia `LayerNormalization`.

> **Note**: `LayerNormalization` ni sawa na `BatchNormalization` iliyojadiliwa katika sehemu ya *Computer Vision* ya njia hii ya kujifunza, lakini husawazisha matokeo ya safu ya awali kwa kila sampuli ya mafunzo kwa kujitegemea, ili kuyafikisha katika kiwango [-1..1].

Matokeo ya safu hii kisha hupitishwa kupitia mtandao wa `Dense` (katika kesi yetu - perceptron ya safu mbili), na matokeo huongezwa kwenye matokeo ya mwisho (ambayo husawazishwa tena).


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Models

**BERT** (Bidirectional Encoder Representations from Transformers) ni mtandao mkubwa sana wa tabaka nyingi wa transformer wenye tabaka 12 kwa *BERT-base*, na 24 kwa *BERT-large*. Modeli hii huanza kwa kufundishwa awali kwenye mkusanyiko mkubwa wa data ya maandishi (WikiPedia + vitabu) kwa kutumia mafunzo yasiyo ya usimamizi (kutabiri maneno yaliyofichwa katika sentensi). Wakati wa mafunzo ya awali, modeli hujifunza kiwango kikubwa cha uelewa wa lugha ambacho kinaweza kutumika na seti nyingine za data kwa kutumia kurekebisha mafunzo. Mchakato huu unaitwa **transfer learning**.

![picha kutoka http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sw/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Kuna aina nyingi za usanifu wa Transformer ikiwa ni pamoja na BERT, DistilBERT, BigBird, OpenGPT3 na nyinginezo ambazo zinaweza kurekebishwa. 

Hebu tuone jinsi tunavyoweza kutumia modeli ya BERT iliyofundishwa awali kutatua tatizo letu la jadi la uainishaji wa mfululizo. Tutakopa wazo na baadhi ya msimbo kutoka [nyaraka rasmi](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Ili kupakia modeli zilizofundishwa awali, tutatumia **Tensorflow hub**. Kwanza, hebu tupakie vectorizer maalum ya BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Ni muhimu kutumia vectorizer sawa na ile ambayo mtandao wa awali ulifundishwa nayo. Pia, BERT vectorizer inarudisha vipengele vitatu:
* `input_word_ids`, ambayo ni mlolongo wa namba za tokeni kwa sentensi ya ingizo
* `input_mask`, inayoonyesha ni sehemu gani ya mlolongo ina ingizo halisi, na ni ipi ni padding. Hii ni sawa na mask inayozalishwa na safu ya `Masking`
* `input_type_ids` hutumika kwa kazi za uundaji wa lugha, na inaruhusu kubainisha sentensi mbili za ingizo katika mlolongo mmoja.

Kisha, tunaweza kuanzisha BERT feature extractor:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Kwa hivyo, safu ya BERT inarudisha matokeo kadhaa muhimu:
* `pooled_output` ni matokeo ya wastani wa alama zote katika mlolongo. Unaweza kuiona kama uwakilishi wa kijasusi wa maana ya mtandao mzima. Hii ni sawa na matokeo ya safu ya `GlobalAveragePooling1D` katika mfano wetu wa awali.
* `sequence_output` ni matokeo ya safu ya mwisho ya transformer (inayolingana na matokeo ya `TransformerBlock` katika mfano wetu hapo juu).
* `encoder_outputs` ni matokeo ya safu zote za transformer. Kwa kuwa tumepakia mfano wa BERT wenye safu 4 (kama unavyoweza kudhani kutoka kwa jina, ambalo lina `4_H`), ina tensors 4. Tensor ya mwisho ni sawa na `sequence_output`.

Sasa tutaelezea mfano wa uainishaji wa mwisho hadi mwisho. Tutatumia *ufafanuzi wa mfano wa kiutendaji*, ambapo tunaelezea ingizo la mfano, kisha tunatoa mfululizo wa maelezo ili kuhesabu matokeo yake. Pia tutafanya uzito wa mfano wa BERT usiweze kufundishwa, na kufundisha tu uainishaji wa mwisho:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Ingawa kuna vigezo vichache vinavyoweza kufundishwa, mchakato ni wa polepole sana kwa sababu kipengele cha uchimbaji cha BERT kinahitaji hesabu nyingi. Inaonekana hatukuweza kufikia usahihi wa kuridhisha, labda kutokana na ukosefu wa mafunzo au ukosefu wa vigezo vya modeli.

Hebu jaribu kufungua uzito wa BERT na kuifundisha pia. Hii inahitaji kiwango kidogo sana cha kujifunza, na pia mkakati wa mafunzo wa uangalifu zaidi na **warmup**, kwa kutumia **AdamW** optimizer. Tutatumia kifurushi cha `tf-models-official` kuunda optimizer:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Kama unavyoona, mafunzo yanaenda polepole sana - lakini unaweza kutaka kujaribu na kufundisha modeli kwa vipindi vichache (5-10) na uone kama unaweza kupata matokeo bora ukilinganisha na mbinu tulizotumia hapo awali.

## Maktaba ya Huggingface Transformers

Njia nyingine maarufu (na rahisi kidogo) ya kutumia modeli za Transformer ni [HuggingFace package](https://github.com/huggingface/), ambayo hutoa vipengele rahisi kwa kazi mbalimbali za NLP. Inapatikana kwa Tensorflow na PyTorch, mfumo mwingine maarufu wa mitandao ya neva.

> **Note**: Ikiwa huna nia ya kuona jinsi maktaba ya Transformers inavyofanya kazi - unaweza kuruka hadi mwisho wa daftari hili, kwa sababu hautaona kitu tofauti sana na kile tulichofanya hapo juu. Tutakuwa tukirudia hatua zile zile za kufundisha modeli ya BERT kwa kutumia maktaba tofauti na modeli kubwa zaidi. Kwa hivyo, mchakato unahusisha mafunzo ya muda mrefu, hivyo unaweza kutaka tu kupitia msimbo.

Hebu tuone jinsi tatizo letu linaweza kutatuliwa kwa kutumia [Huggingface Transformers](http://huggingface.co).


Jambo la kwanza tunalohitaji kufanya ni kuchagua mfano ambao tutatumia. Mbali na baadhi ya mifano iliyojengwa ndani, Huggingface ina [hifadhi ya mifano mtandaoni](https://huggingface.co/models), ambapo unaweza kupata mifano mingi zaidi iliyofunzwa awali na jamii. Mifano yote hiyo inaweza kupakiwa na kutumika kwa kutoa tu jina la mfano. Faili zote za binary zinazohitajika kwa mfano huo zitapakuliwa moja kwa moja.

Wakati fulani utahitaji kupakia mifano yako mwenyewe, ambapo unaweza kubainisha saraka inayojumuisha faili zote husika, ikiwa ni pamoja na vigezo vya tokenizer, faili `config.json` yenye vigezo vya mfano, uzito wa binary, n.k.

Kutoka kwa jina la mfano, tunaweza kuanzisha mfano na tokenizer. Hebu tuanze na tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Kipengele cha `tokenizer` kina kazi ya `encode` ambayo inaweza kutumika moja kwa moja kusimba maandishi:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Tunaweza pia kutumia tokenizer kusimba mlolongo kwa njia inayofaa kwa kupitisha kwa modeli, yaani, ikijumuisha `token_ids`, `input_mask` na mengineyo. Tunaweza pia kubainisha kwamba tunataka Tensorflow tensors kwa kutoa hoja ya `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Katika hali yetu, tutatumia mfano wa awali wa BERT unaoitwa `bert-base-uncased`. *Uncased* inaonyesha kuwa mfano huu haujali herufi kubwa au ndogo.

Wakati wa kufundisha mfano, tunahitaji kutoa mlolongo uliogawanywa katika tokeni kama pembejeo, na kwa hivyo tutatengeneza mchakato wa usindikaji wa data. Kwa kuwa `tokenizer.encode` ni kazi ya Python, tutatumia mbinu ile ile kama katika kitengo cha mwisho kwa kuiita kwa kutumia `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Sasa tunaweza kupakia mfano halisi kwa kutumia kifurushi cha `BertForSequenceClassification`. Hii inahakikisha kwamba mfano wetu tayari una usanifu unaohitajika kwa ajili ya uainishaji, ikijumuisha uainishaji wa mwisho. Utaona ujumbe wa onyo ukisema kwamba uzito wa uainishaji wa mwisho haujaanzishwa, na mfano utahitaji mafunzo ya awali - hilo ni sawa kabisa, kwa sababu ndivyo hasa tunavyotarajia kufanya!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Kama unavyoona kutoka kwa `summary()`, modeli ina karibu vigezo milioni 110! Inawezekana, ikiwa tunataka kazi rahisi ya uainishaji kwenye seti ndogo ya data, hatutaki kufundisha safu ya msingi ya BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Sasa tuko tayari kuanza mafunzo!

> **Note**: Kufundisha mfano wa BERT kwa kiwango kikubwa kunaweza kuchukua muda mwingi! Kwa hivyo, tutaufundisha tu kwa kundi la kwanza la 32. Hii ni kuonyesha tu jinsi mafunzo ya mfano yanavyowekwa. Ikiwa unavutiwa kujaribu mafunzo ya kiwango kikubwa - ondoa tu vigezo vya `steps_per_epoch` na `validation_steps`, na jiandae kusubiri!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Ikiwa utaongeza idadi ya marudio na kusubiri kwa muda wa kutosha, na kufundisha kwa vipindi kadhaa, unaweza kutarajia kwamba uainishaji wa BERT utatupa usahihi bora! Hii ni kwa sababu BERT tayari inaelewa vizuri muundo wa lugha, na tunachohitaji ni kurekebisha kidogo uainishaji wa mwisho. Hata hivyo, kwa sababu BERT ni mfano mkubwa, mchakato mzima wa mafunzo huchukua muda mrefu, na unahitaji nguvu kubwa ya kompyuta! (GPU, na ikiwezekana zaidi ya moja).

> **Note:** Katika mfano wetu, tumekuwa tukitumia moja ya mifano midogo zaidi ya BERT iliyofundishwa awali. Kuna mifano mikubwa zaidi ambayo huenda ikatoa matokeo bora.


## Muhimu

Katika kipengele hiki, tumeangalia miundo ya hivi karibuni ya mifano inayotegemea **transformers**. Tumeitumia kwa kazi yetu ya uainishaji wa maandishi, lakini vivyo hivyo, mifano ya BERT inaweza kutumika kwa uchimbaji wa entiti, kujibu maswali, na kazi nyingine za NLP.

Mifano ya transformer inawakilisha hali ya juu zaidi ya teknolojia ya sasa katika NLP, na mara nyingi inapaswa kuwa suluhisho la kwanza unaloanza kujaribu unapotekeleza suluhisho maalum za NLP. Hata hivyo, kuelewa kanuni za msingi za mitandao ya neva inayorudiwa zilizojadiliwa katika moduli hii ni muhimu sana ikiwa unataka kujenga mifano ya neva ya hali ya juu.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
